In [2]:
!pip install datasets transformers torch torchvision faiss-cpu

In [37]:
!pip install torch torchvision pillow sentence-transformers ftfy git+https://github.com/openai/CLIP.git pandas numpy

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-zmn_k05o
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-zmn_k05o
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.6 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=df14e61bd1d9255299d035b4147f8159c419c1a10194e6a551803073f4dd5fff
  Stored in directory: /tmp/pip-ephem-wheel-cache-etcox2_g/wheels/3f/7c/a4/9b490845988bf7a4db33674d52f709f088f64392063872eb9a
Successfully built clip


In [52]:
import pandas as pd
import numpy as np
import torch
import faiss
from datasets import load_dataset
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import requests
from io import BytesIO
from datetime import datetime
from tqdm import tqdm
import torch
import clip
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
print("Loaded Model")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


...

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

Loaded Model


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Load the dataset of beauty products of Amazon taken from hugging face
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/DL_2025/Project/beauty_products.csv')

In [10]:
# Print the first five rows of the dataset
df.head()

,product_main_category,product_title,product_average_rating,product_rating_number,product_features,product_description,product_price,product_images,product_videos,product_store,product_categories,product_details,parent_asin,product_bought_together,product_subtitle,product_author,Combined_text,prod_imgs
0,All Beauty,"Howard LC0008 Leather Conditioner, 8-Ounce (4-...",4.8,10,[],[],NaN,"{'hi_res': [None, 'https://m.media-amazon.com/...","{'title': [], 'url': [], 'user_id': []}",Howard Products,[],"{""Package Dimensions"": ""7.1 x 5.5 x 3 inches; ...",B01CUPMQZE,NaN,NaN,NaN,"Howard LC0008 Leather Conditioner, 8-Ounce (4-...",['https://m.media-amazon.com/images/I/71i77AuI...
1,All Beauty,Yes to Tomatoes Detoxifying Charcoal Cleanser ...,4.5,3,[],[],NaN,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Yes To,[],"{""Item Form"": ""Powder"", ""Skin Type"": ""Acne Pro...",B076WQZGPM,NaN,NaN,NaN,Yes to Tomatoes Detoxifying Charcoal Cleanser ...,['https://m.media-amazon.com/images/I/71g1lP0p...
2,All Beauty,Eye Patch Black Adult with Tie Band (6 Per Pack),4.4,26,[],[],NaN,"{'hi_res': [None, None], 'large': ['https://m....","{'title': [], 'url': [], 'user_id': []}",Levine Health Products,[],"{""Manufacturer"": ""Levine Health Products""}",B000B658RI,NaN,NaN,NaN,Eye Patch Black Adult with Tie Band (6 Per Pac...,['https://m.media-amazon.com/images/I/31bz+uqz...
3,All Beauty,"Tattoo Eyebrow Stickers, Waterproof Eyebrow, 4...",3.1,102,[],[],NaN,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Cherioll,[],"{""Brand"": ""Cherioll"", ""Item Form"": ""Powder"", ""...",B088FKY3VD,NaN,NaN,NaN,"Tattoo Eyebrow Stickers, Waterproof Eyebrow, 4...",['https://m.media-amazon.com/images/I/71GJhXQG...
4,All Beauty,Precision Plunger Bars for Cartridge Grips – 9...,4.3,7,"['Material: 304 Stainless Steel; Brass tip', '...",['The Precision Plunger Bars are designed to w...,NaN,"{'hi_res': [None], 'large': ['https://m.media-...","{'title': [], 'url': [], 'user_id': []}",Precision,[],"{""UPC"": ""644287689178""}",B07NGFDN6G,NaN,NaN,NaN,Precision Plunger Bars for Cartridge Grips – 9...,['https://m.media-amazon.com/images/I/31TgqAZ8...


In [7]:
# Prints the sahape of the dataset
df.shape

(112578, 18)

In [8]:
# Check the missing or null values
df.isnull().sum()

,0
product_main_category,0
product_title,0
product_average_rating,0
product_rating_number,0
product_features,0
product_description,0
product_price,94875
product_images,0
product_videos,0
product_store,11342


# Generate randomly 50 titles from the dataset. We will use the parts of each of the tiltles or modify them for search in Amazon. For each of the search we will record top 5 products recommended by amazon.

In [13]:
# Randomly sample 50 entries from a specific column
sampled_entries = df['product_title'].sample(n=50, random_state=42)

# Optionally convert to a list
sampled_list = sampled_entries.tolist()

In [14]:
sampled_entries.value_counts()

,count
product_title,
"8 Pieces Nose Rings Bone Stud, Nose Gem 20G comes with 1 Nose Retainer",1
"Vanity Case, Endless Love Lipstick, Corsica Lipshine",1
Nail Files,1
"Bath and Body Works Vanilla Bean Noel Gift Set (Shower Gel, Body Lotion, and Body Splash) Holiday Traditions Collection 2008",1
Leeons Scrunchy Updo Wavy Straight Hair Bun Messy Donut Elastic Chignons Wrap Around Synthetic Ponytail Hairpiece Hair Extension (#2/30),1
Vigorol Mousse Relaxed Hair 12 Ounce (354ml) (2 Pack),1
"Mariska 10A Straight Human Hair Bundles with Closure Brazilian Hair Straight Bundles 100% Unprocessed Human Hair 3 Bundles with Closure for Black Women (16 18 20+14, Straight bundles with closure)",1
"Blitzby 20 Pcs Tattoo Lipstick Cigarette Cotton Swab Lip Stain, Portable Non-Stick Liquid Lipstick, Long-Lasting, and Waterproof Lip Tint. (Red)",1
New Arrival! SOC Animal Cutie Moisturizing Facial Mask (4 Types x2),1


In [58]:
# Load the dataset of recommeded Amazon products from the queries
df_query = pd.read_csv('/content/drive/My Drive/Colab Notebooks/DL_2025/Project/Amazon_recom_queries.csv')

In [59]:
# Print the first 10 rows of the dataset
df_query.head(10)

,Queries,Product_link,Product_title,Image_link,Product_description
0,Nose Rings Bone Stud,https://www.amazon.com/Hoeudjo-Surgical-Zircon...,Hoeudjo Nose Studs Surgical Steel Cubic Zircon...,https://m.media-amazon.com/images/I/618A3KC3Ua...,"MaterialMetal, Rhinestone Metal typeStainless ..."
1,Nose Rings Bone Stud,https://www.amazon.com/Zolure-Surgical-20-30PC...,Zolure 18G 20G 22G Nose Pin Bone Screws Studs ...,https://m.media-amazon.com/images/I/71HH1cIQNe...,"MaterialStainless Steel,Steel,Stone,Diamond Me..."
2,Nose Rings Bone Stud,https://www.amazon.com/Surgical-Stainless-Hypo...,"Cute Nose Rings for Women, Surgical Stainless ...",https://m.media-amazon.com/images/I/51pfB-1xZ8...,MaterialStainless Steel Metal typeStainless St...
3,Nose Rings Bone Stud,https://www.amazon.com/Ocptiy-Surgical-Stainle...,Ocptiy 20G 18G 16G Push in Theadless Nose Ring...,https://m.media-amazon.com/images/I/614DppQpTc...,"MaterialStainless Steel, Cubic Zirconia Metal ..."
4,Nose Rings Bone Stud,https://www.amazon.com/Thunaraz-60pcs-Stainles...,THUNARAZ Straight L Shaped Stainless Steel Nos...,https://m.media-amazon.com/images/I/510aIu8nTZ...,"MaterialStainless Steel,Crystal Metal typeStai..."
5,Love Lipstick,https://www.amazon.com/Clinique-Lasting-Lipsti...,Clinique Clinique Pop Longwear Lipstick | Long...,https://m.media-amazon.com/images/I/61ymji-L-u...,Brand\tClinique Color\tAG Love Pop - Shine Ski...
6,Love Lipstick,https://www.amazon.com/Lightweight-Hydrating-L...,"Glitter Lipstick, Metallic Lipstick For Women,...",https://m.media-amazon.com/images/I/61syzRB80v...,Brand\tGeneric Color\t1# Skin Type\tAll Item F...
7,Love Lipstick,https://www.amazon.com/Lanc%C3%B4me-LAbsolu-Hy...,Lancôme Lancôme L'Absolu Rouge Hydrating Cream...,https://m.media-amazon.com/images/I/61DzzSPs7B...,Brand\tLancôme Color\t337 Blush Classique: coo...
8,Love Lipstick,https://www.amazon.com/Clinique-Dramatically-D...,Clinique Clinique Dramatically Different Lipst...,https://m.media-amazon.com/images/I/61vS53Nfdz...,Brand\tClinique Color\tAL Raspberry Glace 44 S...
9,Love Lipstick,https://www.amazon.com/Lustrous-Lipstick-Vitam...,"Revlon Super Lustrous Lipstick, Creamy Formula...",https://m.media-amazon.com/images/I/71bfF0ljX2...,Brand\tREVLON Color\t777 Vampire Love Skin Typ...


In [60]:
# Shape of the dataset
df_query.shape

(250, 5)

In [61]:
# Check the null values
df_query.isnull().sum()

,0
Queries,0
Product_link,0
Product_title,0
Image_link,0
Product_description,0


In [62]:
# Print the query column
df_query['Queries'].value_counts()

,count
Queries,
Nose Rings Bone Stud,5
Love Lipstick,5
Nail Files,5
"Shower Gel, Body Lotion, and Body Splash",5
Scrunchy Updo Wavy Straight Hair Bun,5
Mousse Relaxed Hair,5
10A Straight Human Hair Bundles with Closure,5
Blitzby Tattoo Lipstick Cigarette Cotton Swab,5
Moisturizing Facial Mask,5


# APPLY CLIP MODEL

### Define the embeddings for text and image

In [63]:
def get_text_embedding(text):
    # Truncate the text to the maximum context length
    text = text[:77]
    # Use clip.tokenize
    text_input = clip.tokenize([text]).to(device)
    with torch.no_grad():
        # Use clip_model.get_text_features()
        text_embedding = clip_model.get_text_features(text_input)
    return text_embedding.cpu().numpy().flatten()

def get_image_embedding(image_url):
    try:
        response = requests.get(image_url)
        image = Image.open(BytesIO(response.content)).convert("RGB")
        # Use clip_processor instead of preprocess
        image_input = clip_processor(images=image, return_tensors="pt").to(device)
        with torch.no_grad():
            # Use clip_model.get_image_features()
            image_embedding = clip_model.get_image_features(image_input.pixel_values)
        return image_embedding.cpu().numpy().flatten()
    except:
        return np.zeros(512)  # Fallback if image fails to load

In [64]:
# Initialize columns for embeddings
df_query["text_embedding"] = None
df_query["image_embedding"] = None
df_query["multimodal_embedding"] = None

# Compute embeddings for each row
for idx, row in tqdm(df_query.iterrows(), total=len(df_query)):
    # Text embedding (query + product title/description)
    combined_text = f"{row['Queries']} | {row['Product_title']} | {row['Product_description']}"
    text_emb = get_text_embedding(combined_text)

    # Image embedding
    image_emb = get_image_embedding(row["Image_link"])

    # Combine embeddings (normalized concatenation)
    multimodal_emb = np.concatenate([text_emb, image_emb])
    multimodal_emb = multimodal_emb / np.linalg.norm(multimodal_emb)  # Normalize

    df_query.at[idx, "text_embedding"] = text_emb.tolist()
    df_query.at[idx, "image_embedding"] = image_emb.tolist()
    df_query.at[idx, "multimodal_embedding"] = multimodal_emb.tolist()

100%|██████████| 250/250 [02:44<00:00,  1.52it/s]


In [68]:
results = []
queries = df_query["Queries"].unique()

for query in queries:
    query_text_emb = get_text_embedding(query)
    query_multimodal_emb = np.concatenate([query_text_emb, np.zeros(512)])  # Placeholder for image
    query_multimodal_emb = query_multimodal_emb / np.linalg.norm(query_multimodal_emb)

    query_products = df_query[df_query["Queries"] == query]
    product_embeddings = np.stack(query_products["multimodal_embedding"].values)

    similarities = cosine_similarity(
        query_multimodal_emb.reshape(1, -1),
        product_embeddings
    )[0]

    for idx, row in query_products.iterrows():
        results.append({
            "Query": query,
            "Product_Title": row["Product_title"],
            "Product_Description": row["Product_description"],
            "Image_Link": row["Image_link"],
            "Similarity_Score": similarities[idx - query_products.index[0]]
        })

results_df = pd.DataFrame(results)

In [71]:
results_df.head()

,Query,Product_Title,Product_Description,Image_Link,Similarity_Score
0,Nose Rings Bone Stud,Hoeudjo Nose Studs Surgical Steel Cubic Zircon...,"MaterialMetal, Rhinestone Metal typeStainless ...",https://m.media-amazon.com/images/I/618A3KC3Ua...,0.468212
1,Nose Rings Bone Stud,Zolure 18G 20G 22G Nose Pin Bone Screws Studs ...,"MaterialStainless Steel,Steel,Stone,Diamond Me...",https://m.media-amazon.com/images/I/71HH1cIQNe...,0.417197
2,Nose Rings Bone Stud,"Cute Nose Rings for Women, Surgical Stainless ...",MaterialStainless Steel Metal typeStainless St...,https://m.media-amazon.com/images/I/51pfB-1xZ8...,0.477182
3,Nose Rings Bone Stud,Ocptiy 20G 18G 16G Push in Theadless Nose Ring...,"MaterialStainless Steel, Cubic Zirconia Metal ...",https://m.media-amazon.com/images/I/614DppQpTc...,0.484552
4,Nose Rings Bone Stud,THUNARAZ Straight L Shaped Stainless Steel Nos...,"MaterialStainless Steel,Crystal Metal typeStai...",https://m.media-amazon.com/images/I/510aIu8nTZ...,0.512862


In [75]:
import altair as alt

# Group by query and calculate the average similarity score
average_similarity = results_df.groupby('Query')['Similarity_Score'].mean().reset_index()

# Create the Altair chart
chart = alt.Chart(average_similarity).mark_bar().encode(
    x='Query',
    y='Similarity_Score'
).properties(
    title='Average Similarity Scores for Each Query',
    width=600,
    height=400
)

chart


alt.Chart(...)

# ZERO SHOT CLASSIFICATION TASK

In [80]:
# Load model and preprocess
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

100%|███████████████████████████████████████| 338M/338M [00:05<00:00, 66.9MiB/s]


In [81]:
# Function to define zero shot classification
def zero_shot_classify(image_url, class_labels, query=None):

    try:
        # Load image
        response = requests.get(image_url, timeout=5)
        image = Image.open(BytesIO(response.content)).convert("RGB")
        image_input = preprocess(image).unsqueeze(0).to(device)

        # Prepare text inputs (combine query with class labels)
        if query:
            text_inputs = torch.cat([clip.tokenize(f"{query}: {label}") for label in class_labels]).to(device)
        else:
            text_inputs = torch.cat([clip.tokenize(label) for label in class_labels]).to(device)

        # Calculate features
        with torch.no_grad():
            image_features = model.encode_image(image_input)
            text_features = model.encode_text(text_inputs)

        # Compute similarity and softmax
        similarity = (image_features @ text_features.T).softmax(dim=-1)
        probs = similarity.cpu().numpy().flatten()

        return {label: float(prob) for label, prob in zip(class_labels, probs)}

    except Exception as e:
        print(f"Error processing {image_url}: {e}")
        return {label: 0.0 for label in class_labels}

In [82]:
class_labels = [
    "matte lipstick",
    "glossy lipstick",
    "metallic lipstick",
    "long-lasting lipstick",
    "vegan lipstick"
]

# Example: Classify an image
image_url = "https://m.media-amazon.com/images/I/61ymji-L-uL._SX522_.jpg"
results = zero_shot_classify(image_url, class_labels, query="Love Lipstick")
print(results)

{'matte lipstick': 0.21335530281066895, 'glossy lipstick': 0.047391377389431, 'metallic lipstick': 0.04698806256055832, 'long-lasting lipstick': 0.519709050655365, 'vegan lipstick': 0.17255611717700958}


In [89]:
# Classify all products for a query
def classify_products(df, query, class_labels):
    df_class = df_query[df_query["Queries"] == query]
    results = []
    for _, row in df_class.iterrows():
        probs = zero_shot_classify(row["Image_link"], class_labels, query)
        results.append({
            "Product_Title": row["Product_title"],
            "Class_Probabilities": probs,
            "Top_Class": max(probs, key=probs.get)
        })
    return pd.DataFrame(results)

# Usage
lipstick_labels = ["matte", "glossy", "metallic", "long-lasting", "vegan"]
classify_products(df, "Love Lipstick", lipstick_labels)

,Product_Title,Class_Probabilities,Top_Class
0,Clinique Clinique Pop Longwear Lipstick | Long...,"{'matte': 0.3266371488571167, 'glossy': 0.0221...",long-lasting
1,"Glitter Lipstick, Metallic Lipstick For Women,...","{'matte': 0.22819970548152924, 'glossy': 0.082...",long-lasting
2,Lancôme Lancôme L'Absolu Rouge Hydrating Cream...,"{'matte': 0.6437317132949829, 'glossy': 0.0143...",matte
3,Clinique Clinique Dramatically Different Lipst...,"{'matte': 0.3837098777294159, 'glossy': 0.0189...",long-lasting
4,"Revlon Super Lustrous Lipstick, Creamy Formula...","{'matte': 0.4977723956108093, 'glossy': 0.1106...",matte


# ZERO SHOT WITH COSINE SIMILARITY

In [90]:
def zero_shot_cosine_similarity(image_url, class_labels, query=None):

    try:
        # Load and preprocess image
        response = requests.get(image_url, timeout=5)
        image = Image.open(BytesIO(response.content)).convert("RGB")
        image_input = preprocess(image).unsqueeze(0).to(device)

        # Prepare text inputs (combine query with labels if provided)
        if query:
            texts = [f"{query}: {label}" for label in class_labels]
        else:
            texts = class_labels
        text_inputs = clip.tokenize(texts).to(device)

        # Get embeddings
        with torch.no_grad():
            image_embedding = model.encode_image(image_input).cpu().numpy()
            text_embeddings = model.encode_text(text_inputs).cpu().numpy()

        # Compute cosine similarity
        similarities = cosine_similarity(image_embedding, text_embeddings)[0]
        return {label: float(sim) for label, sim in zip(class_labels, similarities)}

    except Exception as e:
        print(f"Error processing {image_url}: {e}")
        return {label: 0.0 for label in class_labels}

In [91]:
# For "Love Lipstick" query
lipstick_labels = [
    "matte lipstick",
    "glossy lipstick",
    "metallic lipstick",
    "long-lasting lipstick"
]

# For "Nose Rings Bone Stud" query
nose_ring_labels = [
    "surgical steel nose ring",
    "bone stud nose piercing",
    "cubic zirconia nose stud"
]

In [92]:
# Example 1: Lipstick
image_url = "https://m.media-amazon.com/images/I/61ymji-L-uL._SX522_.jpg"
results = zero_shot_cosine_similarity(image_url, lipstick_labels, query="Love Lipstick")
print(results)

{'matte lipstick': 0.2902916669845581, 'glossy lipstick': 0.2931816577911377, 'metallic lipstick': 0.29459160566329956, 'long-lasting lipstick': 0.30270734429359436}




# Cosine similarity between query and recommended products


In [95]:
def get_query_vs_recommendations_similarity(query, product_title, product_description, image_url):
    """
    Compute cosine similarity between:
    - The search query (text)
    - Each recommended product (text + image)

    Returns:
        dict: {"text_similarity": float, "image_similarity": float, "combined_similarity": float}
    """
    try:
        # Text Embeddings
        with torch.no_grad():
            query_text_embed = model.encode_text(clip.tokenize([query]).to(device))
            product_text = f"{product_title} | {product_description}"
            product_text_embed = model.encode_text(clip.tokenize([product_text], truncate=True).to(device))

            # Image Embedding
            response = requests.get(image_url, timeout=5)
            image = Image.open(BytesIO(response.content)).convert("RGB")
            image_embed = model.encode_image(preprocess(image).unsqueeze(0).to(device))

        # Convert to numpy arrays (detach first)
        query_np = query_text_embed.cpu().detach().numpy()
        product_text_np = product_text_embed.cpu().detach().numpy()
        image_np = image_embed.cpu().detach().numpy()

        # Compute Cosine Similarities
        text_sim = cosine_similarity(query_np, product_text_np)[0][0]
        image_sim = cosine_similarity(query_np, image_np)[0][0]

        # Combined similarity (average of text and image)
        combined_sim = (text_sim + image_sim) / 2

        return {
            "text_similarity": float(text_sim),
            "image_similarity": float(image_sim),
            "combined_similarity": float(combined_sim)
        }

    except Exception as e:
        print(f"Error processing {product_title}: {str(e)}")
        return {
            "text_similarity": 0.0,
            "image_similarity": 0.0,
            "combined_similarity": 0.0
        }

In [96]:
# Load your dataset
df_query = pd.read_csv('/content/drive/My Drive/Colab Notebooks/DL_2025/Project/Amazon_recom_queries.csv')

# Compute similarities for each query-product pair
results = []
for _, row in df_query.iterrows():
    sim_scores = get_query_vs_recommendations_similarity(
        query=row["Queries"],
        product_title=row["Product_title"],
        product_description=row["Product_description"],
        image_url=row["Image_link"]
    )
    results.append({
        "Query": row["Queries"],
        "Product_Title": row["Product_title"],
        "Text_Similarity": sim_scores["text_similarity"],
        "Image_Similarity": sim_scores["image_similarity"],
        "Combined_Similarity": sim_scores["combined_similarity"]
    })

# Convert to DataFrame
similarity_df = pd.DataFrame(results)

In [97]:
similarity_df.head()

,Query,Product_Title,Text_Similarity,Image_Similarity,Combined_Similarity
0,Nose Rings Bone Stud,Hoeudjo Nose Studs Surgical Steel Cubic Zircon...,0.659267,0.281536,0.470401
1,Nose Rings Bone Stud,Zolure 18G 20G 22G Nose Pin Bone Screws Studs ...,0.602259,0.265938,0.434098
2,Nose Rings Bone Stud,"Cute Nose Rings for Women, Surgical Stainless ...",0.737492,0.304784,0.521138
3,Nose Rings Bone Stud,Ocptiy 20G 18G 16G Push in Theadless Nose Ring...,0.623772,0.285833,0.454802
4,Nose Rings Bone Stud,THUNARAZ Straight L Shaped Stainless Steel Nos...,0.708592,0.324887,0.516739
